In [1]:
def read_file(filepath):
    with open(filepath)as f:
        str_text = f.read()
        
    return str_text

In [3]:
#read_file('melville-moby_dick.txt')

In [5]:
import spacy
nlp = spacy.load('en_core_web_sm',disable=['parser','tagger','ner'])

In [6]:
nlp.max_length = 1198623

In [7]:
def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [8]:
d = read_file('moby_dick_four_chapters.txt')

In [9]:
tokens = separate_punc(d)

In [10]:
tokens

['call',
 'me',
 'ishmael',
 'some',
 'years',
 'ago',
 'never',
 'mind',
 'how',
 'long',
 'precisely',
 'having',
 'little',
 'or',
 'no',
 'money',
 'in',
 'my',
 'purse',
 'and',
 'nothing',
 'particular',
 'to',
 'interest',
 'me',
 'on',
 'shore',
 'i',
 'thought',
 'i',
 'would',
 'sail',
 'about',
 'a',
 'little',
 'and',
 'see',
 'the',
 'watery',
 'part',
 'of',
 'the',
 'world',
 'it',
 'is',
 'a',
 'way',
 'i',
 'have',
 'of',
 'driving',
 'off',
 'the',
 'spleen',
 'and',
 'regulating',
 'the',
 'circulation',
 'whenever',
 'i',
 'find',
 'myself',
 'growing',
 'grim',
 'about',
 'the',
 'mouth',
 'whenever',
 'it',
 'is',
 'a',
 'damp',
 'drizzly',
 'november',
 'in',
 'my',
 'soul',
 'whenever',
 'i',
 'find',
 'myself',
 'involuntarily',
 'pausing',
 'before',
 'coffin',
 'warehouses',
 'and',
 'bringing',
 'up',
 'the',
 'rear',
 'of',
 'every',
 'funeral',
 'i',
 'meet',
 'and',
 'especially',
 'whenever',
 'my',
 'hypos',
 'get',
 'such',
 'an',
 'upper',
 'hand',
 '

In [11]:
# We gonna pass 25 words --> network predicts 26th word

In [12]:
train_len = 25+1
text_sequences= []

for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    
    text_sequences.append(seq)

In [14]:
' '.join(text_sequences[1])

'me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore'

In [15]:
from keras.preprocessing.text import Tokenizer

In [16]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)

In [17]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [22]:
#sequences[1] we'hve just converted the words into tokens
#tokenizer.index_word

In [23]:
vocab_size = len(tokenizer.word_counts)

In [26]:
# Now we have to convert the sequences list to a numpy array
import numpy as np

In [27]:
sequences = np.array(sequences)

In [28]:
from keras.utils import to_categorical

In [31]:
X = sequences[:,:-1]

In [32]:
y = sequences[:,-1]

In [33]:
y = to_categorical(y,num_classes = vocab_size + 1)

In [34]:
seq_len = X.shape[1]

In [35]:
X.shape

(11368, 25)

In [36]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding


In [44]:
def create_model(vocab_size,seq_len):
    
    model =Sequential()
    model.add(Embedding(vocab_size,seq_len,input_length=seq_len))
    model.add(LSTM(50,return_sequences=True))
    #model.add(LSTM(50,return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(50,activation='relu'))
    
    model.add(Dense(vocab_size,activation='softmax'))
    
    model.compile(loss = 'categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    model.summary()
    
    return model
    

In [46]:
model = create_model(vocab_size+1,seq_len)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 25, 25)            67750     
_________________________________________________________________
lstm_11 (LSTM)               (None, 25, 50)            15200     
_________________________________________________________________
lstm_12 (LSTM)               (None, 50)                20200     
_________________________________________________________________
dense_7 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_8 (Dense)              (None, 2710)              138210    
Total params: 243,910
Trainable params: 243,910
Non-trainable params: 0
_________________________________________________________________


In [47]:
from pickle import dump,load

In [48]:
model.fit(X,y,batch_size=128,epochs=2,verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
11368/11368 [==============================] - 7s 652us/step - loss: 6.9892 - accuracy: 0.04411s
Epoch 2/2
11368/11368 [==============================] - 5s 471us/step - loss: 6.3737 - accuracy: 0.0527


In [49]:
model.save('my_mobydick_model.h5')

In [51]:
dump(tokenizer,open('my_simpletokenizer','wb'))

In [52]:
from keras.preprocessing.sequence import pad_sequences

In [64]:
def generate_text(model,tokenizer,seq_len,seed_text,num_gen_words):
    output_text = []
    input_text = seed_text
    
    for i in range(num_gen_words):
        
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        pad_encoded = pad_sequences([encoded_text],maxlen=seq_len,truncating = 'pre')
        pred_word_ind = model.predict_classes(pad_encoded,verbose=0)[0]
        pred_word = tokenizer.index_word[pred_word_ind]
        input_text += ' '+pred_word
        output_text.append(pred_word)
    
    return ' '.join(output_text)

In [65]:
import random
random.seed(101)
random_pick = random.randint(0,len(text_sequences))

In [66]:
random_seed_text = text_sequences[random_pick]

In [67]:
seed_text = ' '.join(random_seed_text)

In [68]:
seed_text

'and throwing the clothes to one side he really did this in not only a civil but a really kind and charitable way i stood looking'

In [69]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=25)

'the the the the the the the the the the the the the the the the the the the the the the the the the'

In [70]:
from keras.models import load_model

In [71]:
model = load_model('epochBIG.h5')

In [72]:
tokenizer=load(open('epochBIG','rb'))

In [73]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=25)

"at that stubb ' my frame roman eyes of his own power for the whale 's grain to wrenched progeny for a fever drawn up"